In [1]:
!pip install --upgrade geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.5 MB/s eta 0:00:00


In [2]:
%pip install -U geemap

In [3]:
import os
import ee
import geemap
import pandas as pd

In [4]:
ee.Authenticate()
ee.Initialize(project='ee-wetland')
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
boundaries_level1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
m.addLayer(boundaries, {}, 'Country Boundaries')

In [6]:
# Below command does a filter based on table schema, here we say "ADM0_NAME == 'Kenya'".
# You will get details of table schema in the data catalog.
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
country = boundaries.filter("ADM0_NAME == 'United Arab Emirates'")
provinces = boundaries_level1.filter("ADM0_NAME == 'United Arab Emirates'")
m.addLayer(country, {}, 'Your Country')
# Below command zoom to your country
m.center_object(country)

In [7]:
collection1 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate('2024-04-16', '2024-04-20') \
    .filterBounds(country) \
    .filterMetadata('CLOUD_COVER', 'less_than', 25) \
    .filter(ee.Filter.eq('WRS_PATH', 160)) \
    .filter(ee.Filter.eq('WRS_ROW', 43)) \
    .sort("CLOUD_COVER")


In [8]:
print(collection1.size().getInfo())

1


In [9]:
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

In [30]:
def customRemap(image, lowerLimit, upperLimit, newValue):
  mask = image.gte(lowerLimit).And(image.lt(upperLimit))
  return image.where(mask, newValue)

In [10]:
collection1 = collection1.map(apply_scale_factors)

In [31]:
Map = geemap.Map()
image = collection1.first()

# Compute the Normalized Difference Vegetation Index (NDVI).
nir = image.select('SR_B5');
red = image.select('SR_B4');
green = image.select('SR_B3');
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');
ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI');
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']};

vis_param = {'min': 0,
             'max': 0.3,
             'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
             'gamma': 1.5}
ndwi_clipped = ndwi.clip(country)

Map.addLayer(image, vis_param, "First mage")
Map.addLayer(ndwi_clipped, ndviParams, 'NDWI image');
Map.centerObject(image, 8)

Map

Map(center=[24.550111074794792, 55.18145777622019], controls=(WidgetControl(options=['position', 'transparent_…

In [33]:
floodmask = ndwi_clipped.select('NDWI').gt(0.2);
Map.addLayer(floodmask, {'palette': ['blue', 'lightgreen']}, 'Flood mask');
floodMasked = ndwi_clipped.updateMask(floodmask);
Map.addLayer(floodMasked, {'palette': ['blue']}, 'Flood masked');
flood_remaped = customRemap(floodMasked, 0, 2, 1)
Map.addLayer(flood_remaped, {'palette': ['red']}, 'Flood remapped 19 April');

In [34]:
geemap.ee_export_image_to_drive(
    #floodMasked, filename='floodMasked_19April2024.tif', scale=30, file_per_band=True
    flood_remaped, description="Dubai flood inundation 19 April", folder="share", fileNamePrefix='flood_dubai_19April', scale=30
)

In [14]:
collection2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-04-08', '2023-04-11') \
    .filterBounds(country) \
    .filterMetadata('CLOUD_COVER', 'less_than', 25) \
    .filter(ee.Filter.eq('WRS_PATH', 160)) \
    .filter(ee.Filter.eq('WRS_ROW', 43)) \
    .sort("CLOUD_COVER")

In [15]:
print(collection2.size().getInfo())

1


In [16]:
collection2 = collection2.map(apply_scale_factors)

In [17]:
ap = geemap.Map()
image = collection2.first()

# Compute the Normalized Difference Vegetation Index (NDVI).
nir = image.select('SR_B5');
red = image.select('SR_B4');
green = image.select('SR_B3');
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');
ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI');
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']};

vis_param = {'min': 0,
             'max': 0.3,
             'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
             'gamma': 1.5}
ndvi_clipped = ndwi.clip(country)

Map.addLayer(image, vis_param, "First mage")
Map.addLayer(ndvi_clipped, ndviParams, 'NDVI image');
Map.centerObject(image, 8)

Map

Map(bottom=28455.0, center=[24.55140904327638, 55.20849717388555], controls=(WidgetControl(options=['position'…

In [18]:
floodmask = ndvi_clipped.select('NDWI').gt(0.2);
Map.addLayer(floodmask, {'palette': ['blue', 'lightgreen']}, 'Flood mask');
floodMasked = ndvi_clipped.updateMask(floodmask);
Map.addLayer(floodMasked, {'palette': ['blue']}, 'Flood masked');
flood_remaped = customRemap(floodMasked, 0, 2, 1)
Map.addLayer(flood_remaped, {'palette': ['red']}, 'Before Flood remapped');

In [ ]:
geemap.ee_export_image_to_drive(
    #floodMasked, filename='floodMasked_19April2024.tif', scale=30, file_per_band=True
    flood_remaped, description="Dubai water surface", folder="share", fileNamePrefix='Dubai_Before_flood', scale=30
)